In [ ]:
import re
import os
import time
import json
import random
import requests
import pandas as pd
import local_settings as S
from datetime import datetime, timezone, timedelta
from newspaper import Article, Config
from bs4 import BeautifulSoup

In [ ]:
# CONSTANTS
KEY_IDX = 0
GCSJ_API_KEY = S.GCSJ_API_KEYS[KEY_IDX]
GCSJ_ENGINE_ID = S.GCSJ_ENGINE_IDS[KEY_IDX]

In [ ]:
def fetch_google_results(query):
    search_url = "https://www.googleapis.com/customsearch/v1"
    results = []
    for offset in [1, 11, 21, 31, 41]:
        params = {
            "key": GCSJ_API_KEY,
            "cx": GCSJ_ENGINE_ID,
            "start": offset,
            "lr": 'lang_en',
            "gl": 'us',
            "num": 10,
            "q": query,
        }
        res = requests.get(search_url, params=params)
        res.raise_for_status()
        result = res.json().get("items", [])
        results.extend(result)
    return results

In [ ]:
def get_historical_news(count=20):
    results = []
    with open("./data/completed_dates.csv", "r+") as f:
        from_date = datetime.strptime(f.readlines()[-1].strip(), '%Y-%m-%d')
        for ctr in range(1, count+1):
            dt = from_date + timedelta(days=ctr)
            today = dt.strftime('%Y-%m-%d')
            tomorrow = (dt + timedelta(days=1)).strftime('%Y-%m-%d')
            forbidden = ['youtube.com', 'twitter.com', 'facebook.com', 'instagram.com', 'reddit.com', ]
            forbidden = ' '.join([f'-site:{site}' for site in forbidden])
            query = f'US presidential election 2024 "news" {forbidden} after:{today} before:{tomorrow}'
            items = fetch_google_results(query)
            f.write(today + '\n')

            for item in items:
                item['target'] = today
                date_pattern = r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}(?:\.\d+)?(?:Z|[+-]\d{2}:\d{2})?'
                if matches := re.findall(date_pattern, json.dumps(item)):
                    item['time'] = sorted(matches)[len(matches) // 2]
                obj = {
                    'target': item.get('target', None),
                    'time': item.get('time', None),
                    'link': item.get('link', None),
                }
                results.append(obj)

    results_df = pd.DataFrame(results)
    results_df.to_csv(f"./data/search_results_{datetime.now().timestamp()}.csv", sep='|', index=False)
    return results

In [ ]:
# %%script false --no-raise-error
get_historical_news()